In [1]:
row_sample_rate = 0.5
country_min_sample_cutoff = 500
DUMMY_NA = True
FILL="Mean"

In [2]:
import numpy as np
import pandas as pd
import time
import pickle

from helpers import clone_drop_and_convert, clone_and_convert_multiple_choice_columns, clone_impute_data, perf_measures, round_decision, reduce_dataframe_using_min_non_null, get_coefficent_weights
from models import apply_linear_regression_model
from predict_country import predict_country

# Developer note: If you modify the modules above, the changes to those modules will not be reflected until you 
# re-execute this cell, specifically the reload commands
# TODO this doesn't work yet
# import reload
# reload(clone_drop_and_convert)
# reload(models)

pd.options.display.max_rows = 2000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = 255

script_start = time.time()

raw_df = pd.read_csv('./assets/survey_results_public.csv')
schema = pd.read_csv('./assets/survey_results_schema.csv')

In [3]:
start = time.time()
df = clone_and_convert_multiple_choice_columns(clone_drop_and_convert(raw_df))
end = time.time()
print('convert multi choice time %s' % (end - start))
print("convert multi choice dataset has %s rows and %s columns" % (df.shape[0], df.shape[1]))

full_country = df.sample(frac=row_sample_rate, axis=0)

start = time.time()
full_country_imputed = clone_impute_data(full_country, dummy_na=True, fill_technique=FILL)
end = time.time()
print('full_country impute time %s' % (end - start))
print("full country imputed dataset has %s rows and %s columns" % (full_country_imputed.shape[0], full_country_imputed.shape[1]))

convert multi choice time 36.663614988327026
convert multi choice dataset has 64461 rows and 369 columns
full_country impute time 0.3745298385620117
full country imputed dataset has 32230 rows and 454 columns


In [4]:
countries = full_country['Country'].value_counts()
countries_with_enough_samples = countries[countries > (1000 * row_sample_rate)].index.values
countries_with_enough_samples

array(['United States', 'India', 'United Kingdom', 'Germany', 'Canada',
       'France', 'Brazil', 'Netherlands', 'Poland', 'Australia', 'Italy',
       'Russian Federation', 'Spain'], dtype=object)

In [5]:
#cutoffs here pertains to the number of missing values allowed in the used columns.
#Therefore, lower values for the cutoff provides more predictors in the model.

# cutoffs = [5000, 3500, 2500, 1000, 100, 50, 30, 25, 0]
adjust_cutoff_with_sample_rate = lambda cutoff: cutoff * row_sample_rate
#cutoffs = map(adjust_cutoff_with_sample_rate, [5000, 2500, 1000, 500, 250, 100])
cutoffs = [5000, 2500, 1000, 500, 250, 100]

results = []
for country in countries_with_enough_samples:
    country_results = predict_country( \
        rowset_label='full_country_%s' % country, \
        columnset_label='all_numeric_categerical', \
        imputed_df=full_country_imputed, \
        cutoffs=cutoffs, \
        country_to_classify=country, \
        row_sample_rate=row_sample_rate \
    )
    results.extend(country_results)

rowset_label=full_country_United States, columnset_label=all_numeric_categerical, cutoff=5000 yield columns=193, ACC=0.8330747750542973, in duration=208.018s
rowset_label=full_country_United States, columnset_label=all_numeric_categerical, cutoff=2500 yield columns=293, ACC=0.8383493639466336, in duration=303.245s
rowset_label=full_country_United States, columnset_label=all_numeric_categerical, cutoff=1000 yield columns=381, ACC=0.8413486399834522, in duration=409.004s
rowset_label=full_country_United States, columnset_label=all_numeric_categerical, cutoff=500 yield columns=422, ACC=0.84269314303444, in duration=432.132s
rowset_label=full_country_United States, columnset_label=all_numeric_categerical, cutoff=250 yield columns=442, ACC=0.8423828730995967, in duration=463.326s
rowset_label=full_country_United States, columnset_label=all_numeric_categerical, cutoff=100 yield columns=451, ACC=0.8415554866066811, in duration=477.525s
rowset_label=full_country_India, columnset_label=all_nume

In [6]:
# result_df = pd.DataFrame(data=results).set_index(['rowset_label', 'columnset_label', 'country', 'cutoff'])
result_df = pd.DataFrame(data=results).set_index(['country', 'cutoff'])
for to_round in ['TPR', 'TNR', 'PPV', 'NPV', 'ACC', 'duration']:
    result_df[to_round] = result_df[to_round].map(lambda x: round(x,3))

result_df.drop(columns=['X_columns', 'coefficients']).sort_values('ACC')

with open('./pickles/results.pkl', 'wb') as f:
    pickle.dump(result_df, f)

In [7]:
script_end = time.time()
print('script duration %s' % end - start)

TypeError: unsupported operand type(s) for -: 'str' and 'float'